In [ ]:
import geopandas as gpd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import GroupKFold
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    cohen_kappa_score,
)

# =============================================================================
# 1. Load and preprocess geospatial data
# =============================================================================
mesh = gpd.read_file("/home/stagiaire/Téléchargements/PR/D/mesh_rj.shp")
mesh["label"] = np.where(
    (mesh["vegetation"] <= 0.95)
    & (mesh["ghsl"] >= 0.5)
    & (mesh["osm"] <= 0.1)
    & (mesh["favelas"] > 0.9),
    1,
    np.where(
        (mesh["vegetation"] <= 0.95)
        & (mesh["ghsl"] >= 0.5)
        & (mesh["osm"] <= 0.1)
        & (mesh["favelas"] == 0),
        0,
        np.nan,
    ),
)
dataset = mesh[mesh["label"].notna()].copy()

zones = gpd.read_file("/home/stagiaire/Téléchargements/PR/D/zones.shp")
dataset["centroid"] = dataset.geometry.centroid
points_zones = gpd.sjoin(
    dataset.set_geometry("centroid"),
    zones[["fid", "geometry"]],
    how="left",
    predicate="within",
)
dataset["zone"] = points_zones["fid"]
dataset.drop(columns=["centroid"], inplace=True)
dataset = dataset[dataset["zone"].notna()].reset_index(drop=True)

# =============================================================================
# 2. Build feature matrix including up to 8 neighbors
# =============================================================================
# Identify all pairs of intersecting cells (including corner-touching)
neighbors_df = gpd.sjoin(
    dataset, dataset, how="inner", predicate="intersects",
    lsuffix="left", rsuffix="right"
)
neighbors_df = neighbors_df[neighbors_df.index != neighbors_df["index_right"]]

feature_cols = [
    "vegetation", "slope", "profile_co", "entropy",
    "nodes", "roads", "mean_conne", "min_connex", "max_connex",
]
num_features = len(feature_cols)
max_neighbors = 8

X_list, y_list, groups_list = [], [], []
cell_features = {
    int(r["id"]): r[feature_cols].values.astype(np.float32)
    for _, r in dataset.iterrows()
}

for _, row in dataset.iterrows():
    cell_id = int(row["id"])
    groups_list.append(int(row["zone"]))

    # Start with the 9 features of the target cell
    feat_vec = list(cell_features[cell_id])

    # Add up to 8 neighbors’ features
    nbrs = neighbors_df[
        neighbors_df["id_left"] == cell_id
    ].head(max_neighbors)
    for _, nbr in nbrs.iterrows():
        feat_vec.extend(cell_features[int(nbr["id_right"])])

    # Zero-pad if fewer than 8 neighbors
    while len(feat_vec) < (1 + max_neighbors) * num_features:
        feat_vec.extend([0.0] * num_features)

    X_list.append(feat_vec)
    y_list.append(int(row["label"]))

X = torch.tensor(np.array(X_list, dtype=np.float32))
y = torch.tensor(np.array(y_list, dtype=np.int64))
groups = np.array(groups_list)

print("X shape:", X.shape)
print("Number of examples:", X.shape[0])

# =============================================================================
# 3. Undersampling function
# =============================================================================
def undersample(idx, labels):
    """Random undersample to balance binary classes within idx."""
    idx = np.array(idx)
    labs = labels[idx].numpy()
    c0, c1 = idx[labs == 0], idx[labs == 1]
    if len(c0) == 0 or len(c1) == 0:
        return idx
    m = min(len(c0), len(c1))
    res = np.concatenate([
        np.random.choice(c0, m, replace=False),
        np.random.choice(c1, m, replace=False),
    ])
    np.random.shuffle(res)
    return res

# =============================================================================
# 4. Define MLP model
# =============================================================================
class MLP(nn.Module):
    """Single-hidden-layer MLP for binary classification."""
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))


# =============================================================================
# 5. Train and evaluate function
# =============================================================================
def train_and_evaluate_with_neighbors(train_idx, test_idx):
    """Train and evaluate the MLP using neighbor-augmented features."""
    train_idx_bal = undersample(train_idx, y)
    test_idx_bal = undersample(test_idx, y)

    X_train, y_train = X[train_idx_bal], y[train_idx_bal]
    X_test, y_test = X[test_idx_bal], y[test_idx_bal]

    train_loader = DataLoader(
        TensorDataset(X_train, y_train), batch_size=32, shuffle=True
    )
    test_loader = DataLoader(
        TensorDataset(X_test, y_test), batch_size=32, shuffle=False
    )

    model = MLP(
        input_dim=X.shape[1],
        hidden_dim=64,
        output_dim=2,
    ).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(1, 401):
        for bx, by in train_loader:
            bx, by = bx.to(device), by.to(device)
            optimizer.zero_grad()
            loss = criterion(model(bx), by)
            loss.backward()
            optimizer.step()
        if epoch % 100 == 0:
            print(f"Epoch {epoch:03d} complete")

    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for bx, by in test_loader:
            out = model(bx.to(device)).argmax(dim=1).cpu()
            preds.append(out)
            trues.append(by)

    preds = torch.cat(preds).numpy()
    trues = torch.cat(trues).numpy()

    p = precision_score(trues, preds, zero_division=0)
    r = recall_score(trues, preds, zero_division=0)
    f1 = f1_score(trues, preds, zero_division=0)
    k = cohen_kappa_score(trues, preds)
    print(f"P:{p:.3f} R:{r:.3f} F1:{f1:.3f} K:{k:.3f}")
    return p, r, f1, k


# =============================================================================
# 6. Serial spatial cross-validation (10 × 5 folds)
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

n_iters = 10
n_folds = 5
metrics_iters = np.zeros((n_iters, n_folds, 4))  # P, R, F1, Kappa

for it in range(n_iters):
    print(f"\n=== Iteration {it + 1}/{n_iters} ===")
    gkf = GroupKFold(n_splits=n_folds)
    for fold, (tr_idx, te_idx) in enumerate(
        gkf.split(X.numpy(), y.numpy(), groups),
        start=1
    ):
        print(f"\n-- Fold {fold} --")
        metrics_iters[it, fold - 1] = train_and_evaluate_with_neighbors(tr_idx, te_idx)

# =============================================================================
# 7. Aggregate and print results
# =============================================================================
mean_per_fold = metrics_iters.mean(axis=0)
std_per_fold = metrics_iters.std(axis=0)
global_mean = mean_per_fold.mean(axis=0)
global_std = mean_per_fold.std(axis=0)

print("\n--- Per-fold averages ---")
for f in range(n_folds):
    print(
        f"Fold {f+1}: "
        f"P {mean_per_fold[f,0]:.3f}±{std_per_fold[f,0]:.3f}, "
        f"R {mean_per_fold[f,1]:.3f}±{std_per_fold[f,1]:.3f}, "
        f"F1 {mean_per_fold[f,2]:.3f}±{std_per_fold[f,2]:.3f}, "
        f"K {mean_per_fold[f,3]:.3f}±{std_per_fold[f,3]:.3f}"
    )

print(
    "\n--- Global averages ---\n"
    f"P {global_mean[0]:.3f}±{global_std[0]:.3f}, "
    f"R {global_mean[1]:.3f}±{global_std[1]:.3f}, "
    f"F1 {global_mean[2]:.3f}±{global_std[2]:.3f}, "
    f"K {global_mean[3]:.3f}±{global_std[3]:.3f}"
)
